In [ ]:
# GitHub에서 'nsmc. (Naver Sentiment Movie Corpus) 레포지토리 클론 -> 데이터로 사용될 것임
!git clone https://github.com/e9t/nsmc/

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1 (from 1)
Receiving objects: 100% (14763/14763), 56.19 MiB | 17.51 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [ ]:
# 'konlpy'패키지 설치 -> 'konlpy'는 한국어 자연어처리를 위한 파이썬 라이브러리로 형태소 분석 수행 가능
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 31.7 MB/s eta 0:00:00


In [ ]:
# MeCab-ko라는 한국어 형태소 분석기를 설치하기 위해 GitHub에서 레포지토리를 클론하고 설치 스크립트를 실행 -> 구글 코랩 환경에서 MeCab-ko를 설정하여 한국어 텍스트의 형태소 분석을 쉽게 할 수 있게 함.
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 3.67 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2024-08-17 05:25:54--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.142.24, 104.192.142.26, 104.192.142.25, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-dispositi

In [ ]:
# 'LabelEncoder'를 사용하여 ["개","강아지","호랑이","개","고양이"] 리스트의 텍스트 레이블을 숫자형 레이블로 변환. 텍스트 데이터를 모델에 사용할 수 있는 숫자형 데이터로 변환하기 위함.(개 강아지..등은 LabelEncoder의 기능을 보여주기 위한 예시임)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
result= le.fit_transform(["개","강아지","호랑이","개","고양이"])
result

array([1, 0, 3, 1, 2])

In [ ]:
import pandas as pd # csv 로드
from sklearn.model_selection import train_test_split # data 8:2
from sklearn.preprocessing import LabelEncoder # 텍스트 -> 숫자형 바꿀때
from torchtext.data.utils import get_tokenizer # 토크나이저
from torchtext.vocab import build_vocab_from_iterator # 단어사전
 # 단어사전
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import warnings
import torchtext

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
# torchtext deprecation warning 무시
torchtext.disable_torchtext_deprecation_warning()

In [ ]:
# 데이터 로딩(Naver Sentiment Movie Corpus(NSMC) 데이터셋을 탭으로 구분된 파일로부터 불러오고 데이터프레임의 컬럼 이름을 설정)
df = pd.read_csv('/content/nsmc/ratings_train.txt', delimiter='\t')
df.columns = ['id', 'document', 'label']

In [ ]:
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
df.info() # 데이터 프레임 요약 정보를 출력하여 데이터셋의 크기, 데이터타입, 결측치 여부 등 주요 정보를 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
# NaN 값을 빈 문자열로 변환하여 오류 방
df['document'] = df['document'].astype(str)

In [ ]:
df.info() # 데이터프레임의 요약 정보 다시 출력하여 데이터 셋 정보 확인 및 이후 작업 준비

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  150000 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
# 학습용 데이터 80%, 검증용 데이터 20%로 분할
train_df, valid_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42) # random_state는 재현가능한 결과를 위해 설정됨
train_df

,id,document,label
104025,7757635,나의 티모를 더럽히지마,0
5415,9554776,배비장전을 독특하게 각색한 의외로 재미있게 본 영화. 배우들의 연기력과 사극톤이 부...,1
75612,9623284,아기와의 교감도 없어 보이는 미혼모란 탈을 쓴 여자주인공의 행태가 문제인 드라마. ...,0
138169,10162069,아무 생각없이 평점 안보고 영화를 보다가 마지막 엔딩에서 소름 돋는걸 느끼는 내모습...,1
87184,8472187,혼자서 다 할거면 제자는 왜 키우는가.,0
...,...,...,...
119879,4992047,아...뭐이런 ㅄ영화가 다있지? 미치겠다 돈이랑시간이너무아까워서ㅠㅠ개허접졸작ㅡㅡ,0
103694,6402958,유재석없었으면 0점인데 유재석때문에 1점이라도 준다,0
131932,5781720,"아 ...장난함? ㅡ,ㅡ영화만든뜻은 알겠는데 제발 이렇게 영화 만들지 말자",0
146867,9838001,세월이 흘러도 명작은 명작이다!!,1


In [ ]:
# Mecab 형태소 분석기 초기화 및 이를 사용하는 사용자 정의 토크나이저 함수 정의 -> 한국어 텍스트를 분석하기 위해 형태소 단위로 텍스트를 분리하는 함수가 필요하기 때문에 'Mecab'을 사용하여 토큰화 수행
from konlpy.tag import Mecab
from torchtext.data.utils import get_tokenizer

mecab = Mecab()

def custom_tokenizer(text):
  return mecab.morphs(text)

In [ ]:
# 사용자 정의 토크나이저를 'torchtext'의 'get_tokenizer' 함수와 연결 -> 코드를 보다 일관되고 유지보수하기 쉽게 하기 위해 사용자 정의 토크나이저를 표준화된 방식으로 불러옴
tokenizer = get_tokenizer(custom_tokenizer)

In [ ]:
# 주어진 데이터에서 토큰을 생성하는 제너레이터 함수를 정의하고 이를 통해 단어 사전을 생성
def yield_tokens(data_iter): # data_iter에서 텍스트 데이터를 순차적으로 받아와 토큰화된 결과를 생성해내는 제너레이터 함수
  for text in data_iter:
    yield tokenizer(text) # text를 토크나이저 함수로 처리한 후 결과를 제너레이터의 출력으로 반환

vocab = build_vocab_from_iterator(yield_tokens(df['document']), specials=['<unk>']) # yield_tokens 제너레이터를 통해 df['document'] 컬럼의 모든 텍스트에서 단어 사전을 구축
# 사전에 <unk>를 특별 토큰으로 추가하여 <unk>에 지정된 단어는 0이 됨

vocab.set_default_index(vocab['<unk>']) # 사전에 없는 단어는 <unk>로 지

In [ ]:
vocab.get_stoi() # 'vocab'의 'string to index' 매핑 반환

{'￣∇￣)': 54044,
 '＾＾': 54041,
 '！！！': 54038,
 '律': 54036,
 '힛처': 54029,
 '힘준': 54028,
 '힘써서': 54025,
 '힘든가': 54022,
 '힘내기': 54021,
 '힘겨웠': 54020,
 '힘겨워': 54019,
 '힐신': 54017,
 '힐러리스웽크': 54016,
 '힉콕스': 54012,
 '히히히히히': 54010,
 '히포크라테스': 54008,
 '히치': 54005,
 '히지': 54004,
 '히죽히죽': 54003,
 '히잉': 54001,
 '히스토리': 53998,
 '히사코': 53997,
 '히브리': 53994,
 '히무로': 53992,
 '히메나': 53990,
 '히마': 53989,
 '히로시마': 53987,
 '히로미': 53985,
 '히뜩': 53983,
 '히나': 53979,
 '흳': 53978,
 '흰개': 53976,
 '희재': 53975,
 '희기': 53965,
 '희곡': 53964,
 '흩어져서': 53959,
 '흩뿌렸': 53957,
 '흥겨워서': 53948,
 '흥건': 53947,
 '흠좀무': 53946,
 '흙모래': 53943,
 '흘쩍': 53942,
 '흘러야': 53938,
 '흘러버렸': 53937,
 '흘러내리': 53935,
 '흘러나왔': 53934,
 '흘러나와': 53933,
 '흘깃': 53931,
 '흔했': 53930,
 '흔해져서': 53929,
 '흔해빠졌': 53928,
 '흔한데': 53927,
 '흔쾌히': 53926,
 '흔쾌': 53925,
 '흔이': 53924,
 '흑흑흑흑': 53920,
 '흑적색': 53917,
 '흑암': 53914,
 '흐흐흐흐': 53907,
 '흐트림': 53906,
 '흐트러': 53904,
 '흐엉': 53903,
 '흐린다는': 53901,
 '흐른다는': 53897,
 '흐륵': 53896,
 '흐드러지': 53894,
 '흐느적거리': 53

In [ ]:
vocab(tokenizer("▩")) # 토큰화된 텍스트 "▩"를 단어사전을 통해 인덱스 값으로 변환

[0]

In [ ]:
torch.tensor(vocab(tokenizer("안녕 나는 자연어 처리 개발중이야"))) # 안녕 나는 자연어 처리 개발중이야 라는 문장을 토큰화하고 단어 사전을 통해 각 토큰을 인덱스로 변환한 후, 이를 텐서로 변환

tensor([3100,   27,    3,    0, 2714, 3243,  110,    2,  121])

In [ ]:
# 데이터셋 클래스 : 데이터를 불러오거나 정의, 전처리하는 클래스함수
# 유지보수
class ReviewDataset(Dataset): # 텍스트 데이터를 모델에 공급할 수 있도록 정의된 것으로 데이터를 불러오고 전처리하는 기능을 포함
  def __init__(self, dataframe, vocab, tokenizer): # 초기화 메서드로 데이터프레임, 단어사전, 토크나이저를 인자로 받아 클래스변수로 저장
    self.data = dataframe # 리뷰 텍스트와 레이블을 포함한 데이터프레임을 저장
    self.vocab = vocab # 단어사전 객체. 토큰화된 단어를 인덱스로 변환하는데에 쓰임.
    self.tokenizer = tokenizer # 텍스트를 단어 또는 형태소 단위로 분리하는 토크나이저 함수(여기서는 MeCab이 쓰임)

  def __len__(self): # 데이터셋의 크기를 나타내어 데이터 배치를 나누는 작업에 사용
    return len(self.data)

  def __getitem__(self, idx): # 데이터를 불러올 때 반응하는 함수 argument로 몇 번째 데이터를 불러올건지 인덱스를 받음
    text = self.data.iloc[idx]['document'] # 받은 행 인덱스 (몇번째 데이터)의 리뷰데이터 선택
    label = self.data.iloc[idx]['label'] # 받은 행 인덱스 (몇 번째 데이터)의 레이블 선택
    tokenized_text = self.tokenizer(text) # 가져온 idx번째 리뷰데이터를 토크나이저를 통해 단어 또는 형태소 단위로 분리. 텍스트가 토큰의 리스트로 변환
    numerical_text = torch.tensor(self.vocab(tokenized_text)) # 토큰화된 단어 리스트를 단어 사전을 이용해 각 단어를 고유한 숫자 인덱스로 변환. 이를 PyTorch 텐서로 변환
    return numerical_text, torch.tensor(label) # 숫자로 변환된 텍스트와 정답 레이블 반환

In [ ]:
# 학습, 검증용 데이터셋을 ReivewDataset 클래스를 사용해 생성
train_dataset = ReviewDataset(train_df, vocab, tokenizer)
valid_dataset = ReviewDataset(valid_df, vocab, tokenizer)

In [ ]:
import torch

sequence_batch = [
    torch.tensor([1,2,3]),
    torch.tensor([1,2,]),
    torch.tensor([1])
]

torch.nn.utils.rnn.pad_sequence(sequence_batch, batch_first=True) # pad_sequence를 통해 길이가 다른 시퀀스를 동일한 길이로 맞춰줌. batch_first=True는 배치 차원이 첫 번째 차원으로 오도록 설정

tensor([[1, 2, 3],
        [1, 2, 0],
        [1, 0, 0]])

In [ ]:
def collate_fn(batch): # 콜백함수-배치 데이터를 패딩처리하고 텍스트와 레이블을 반환
  texts, labels = zip(*batch) # 배치 내의 각 데이터를 언패킹하여 텍스트와 레이블을 분리
  texts = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True) # 텍스트 데이터는 길이가 다를 수 있으므로 pad_sequence를 사용해 모든 텍스트 시퀀스 길이를 맞춰줌
  labels = torch.stack(labels) # 레이블들을 스택하여 하나의 텐서로 만듦

  return texts, labels

# collate_fn을 사용하는 DataLoader는 배치 데이터를 로드할 때 패딩과 정렬을 자동으로 처리해줌
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=2, collate_fn=collate_fn)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score # 모델 성능 측정 라이브러리

# RNN 모델 정의
class RNNClassifier(nn.Module): # PyTorch의 nn.Module을 상속받아 신경망 모듈로 구현
  def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1): # 모델의 각 계층(embedding, RNN, fully connected layer)을 정의하고 초기화
  # vocab_size는 모델이 처리할 수 있는 전체 단어사전의 크기, embed_size는 단어를 임베딩할 벡터의 차원 수, hidden size는 RNN의 은닉 상태 벡터의 크기, num_layers는 RNN 계층의 수
    super(RNNClassifier, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size) # 임베딩 계층을 정의. 단어 인덱스를 임베딩 벡터로 변환하여 각 단어를 고정된 차원의 벡터로 표현
    self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True) # RNN 계층 정의. 'embed_size' 크기의 입력을 받아 'hidden_size'크기의 은닉 상태 출력 'batch_first=True'는 입력 데이터의 첫번째 차원이 배치 크기임을 의미
    self.fc = nn.Linear(hidden_size, 1) # fully connected layer 정의. RNN의 최종 은닉 상태를 받아 이진분류를 위한 출력 생성(이진분류를 위해 output_size를 1로 설정)

  def forward(self,x): # 입력 데이터를 받아 순전파를 통해 최종 출력값 계산
    x = self.embedding(x) # 입력 텍스트 데이터를 임베딩 벡터로 변환. x는 이제 각 단어가 embed_size 크기의 벡터로 표현된 형태
    h, _ = self.rnn(x) # 임베딩된 벡터 시퀀스를 RNN에 입력하여 각 타임스텝에 대한 은닉 상태를 계산. h는 모든 타임스텝의 은닉 상태를 포함하며, h[:, -1, :]은 시퀀스의 마지막 타임스텝의 은닉 상태를 가져옴.
    out = self.fc(h[:, -1, :]) # 마지막 타임스텝의 은닉상태를 받아 완전 연결계층을 통해 이진분류결과를 계산. 이 값은 로짓이며 이후 시그모이드 함수 등을 통해 확룰로 변환
    return out # 모델의 최종 출력값 반환. 분류를 위해 사용할 수 있음.

In [ ]:
# 하이퍼파라미터 설정
embed_size = 100 # 단어 임베딩 벡터의 크기. 각 단어는 이 크기의 벡터로 변환됨.
hidden_size = 128 # RNN의 은닉 상태 크기. RNN이 처리하는 정보의 크기 결정
output_size = 1 # 모델의 출력 크기. 이진 분류를 위해 1로 설정
num_layers = 1 # RNN 층의 수
learning_rate = 0.001 # 옵티마이저의 학습률
num_epochs = 5 # 학습과정에서 전체 데이터셋을 몇 번 반복할지 설정

# 모델, 손실함수, 옵티마이저 초기화
device = 'cuda' if torch.cuda.is_available() else 'cpu' # 학습을 GPU에서 할 수 있는지 확인하고 가능하면 CUDA 사용. 그렇지 않으면 CPU 사용
model = RNNClassifier(len(vocab), embed_size, hidden_size, num_layers).to(device) # 'RNNClassifier'클래스를 이용해 모델을 생성하고, 이를 'device'로 이동시킴(GPU 또는 CPU)
criterion = nn.BCEWithLogitsLoss() # 이진 분류를 위한 손실함수로 BCEWithLogitsLoss를 사용. 이 함수는 로짓(logit) 값과 실제 레이블 사이의 손실 계산
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # 모델의 파라미터를 업데이트하기 위해 Adam 옵티마이저 사용. Adam은 학습률을 자동으로 조정하여 빠르게 수렴하도록 도와줌

In [ ]:
# 훈련 함수(모델을 학습시키는 함수)
def train():
  model.train() # 모델을 학습모드로 설정. 이는 드롭아웃 등의 학습 시 동작을 활성화(추론속도를 높이기 위함)
  for epoch in range(num_epochs): # 지정된 에포크 수만큼 학습을 반복
    total_loss = 0 # total loss를 변수로 선언
    for inputs, labels in train_loader: # 학습 데이터셋에서 배치별로 데이터를 불러옴
      inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float) # 레이블을 float으로 변환, 데이터를 GPU 또는 CPU로 이동
      optimizer.zero_grad() # 이전 배치의 그래디언트를 초기화
      outputs = model(inputs).squeeze() # 모델에 입력 데이터를 전달하여 예측 결과를 얻음. 차원축소
      loss = criterion(outputs, labels) # 예측값과 실제값 사이의 손실 계산
      loss.backward() # 손실을 역전파하여 그래디언트 계산
      optimizer.step() # 옵티마이저를 사용해 모델의 파라미터 업데이트
      total_loss += loss.item() # 배치의 손실값을 누적
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}') # 각 에포크가 끝날 때 평균 손실 값을 출력

# 검증 함수
def evaluate(loader):
  model.eval() # 모델을 평가모드로 설정. 평가모드에서는 드롭아웃 등이 비활성화
  # 드롭아웃은 신경망이 과적합(overfitting)하는 것을 방지하기 위해 사용되는 기법. 학습 중 무작위로 일부 뉴런을 비활성화하여 학습을 진행
  all_labels = [] # 실제 레이블을 저장할 리스트 초기화
  all_preds = [] # 예측값을 저장할 리스트 초기화
  with torch.no_grad(): # 평가과정에서는 그래디언트를 계산하지 않도록 설정(메모리, 연산 절약)
    for inputs, labels in loader: # 검증 데이터셋에서 배치 단위로 데이터를 불러옴
      inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float) # 데이터를 GPU 혹은 CPU로 이동시키고 레이블을 float 형식으로 변환
      outputs = model(inputs).squeeze() # 모델에 입력을 전달하여 예측 결과를 얻고, 차원을 축소
      preds = (torch.sigmoid(outputs) > 0.5).float() # sigmoid함수를 통해 로짓을 확률로 변환한 뒤 임계값 0.5를 기준으로 이진예측 만듦
      all_labels.extend(labels.cpu().numpy()) # 실제 레이블을 리스트에 추가
      all_preds.extend(preds.cpu().numpy()) # 예측값을 리스트에 추가
  return accuracy_score(all_labels, all_preds) # 모든 예측과 실제 레이블을 비교하여 정확도 계산

# 모델 훈련 및 검증
train() # train 함수 호출하여 모델 훈련
valid_accuracy = evaluate(valid_loader) # 검증 데이터셋을 사용해 모델의 정확도 계산
print(f'Validation Accuracy: {valid_accuracy:.4f}') # 검증 정확도 출력

Epoch [1/5], Loss: 0.5690
Epoch [2/5], Loss: 0.4785
Epoch [3/5], Loss: 0.4536
Epoch [4/5], Loss: 0.4312
Epoch [5/5], Loss: 0.4268
Validation Accuracy: 0.8020


### **`model.train()`의 역할**

- **기능:** PyTorch에서 `model.train()`은 모델을 **학습 모드(Training Mode)**로 전환하는 역할. 이는 모델이 학습할 때 필요한 동작들을 활성화하는 중요한 함수
  
### **학습 모드와 평가 모드의 차이**

1. **학습 모드 (`train` 모드)**: `model.train()`에 의해 활성화
2. **평가 모드 (`eval` 모드)**: `model.eval()`에 의해 활성화

이 두 모드는 주로 **드롭아웃** 및 **배치 정규화(Batch Normalization)**와 관련된 차이를 만듦.

#### **1. 드롭아웃(Dropout)**
드롭아웃은 신경망이 과적합(overfitting)하는 것을 방지하기 위해 사용되는 기법으로 학습 중 무작위로 일부 뉴런을 비활성화하여 학습.
- **`train` 모드에서:** 드롭아웃이 활성화. 학습 과정에서 특정 뉴런들이 무작위로 비활성화. 이를 통해 신경망이 너무 특정 패턴에만 집중하지 않도록 방지
- **`eval` 모드에서:** 드롭아웃이 비활성화. 모든 뉴런이 활성화된 상태에서 예측을 수행. 평가 및 예측 과정에서는 불확실성을 최소화하고 정확한 결과를 얻기 위해 드롭아웃이 사용되지 않음.

#### **2. 배치 정규화(Batch Normalization)**
배치 정규화는 각 미니배치의 통계를 사용하여 입력 데이터를 정규화하는 과정
- **`train` 모드에서:** 배치의 통계값(평균과 분산)을 사용하여 데이터가 정규화. 또한, 배치 정규화의 이동 평균과 분산 값이 업데이트됨.
- **`eval` 모드에서:** 학습 중에 계산된 전체 데이터셋에 대한 이동 평균 및 분산 값을 사용하여 정규화. 예측 시 안정된 결과 보장.

In [ ]:
# 인퍼런스 예제(입력 텍스트에 대해 예측을 수행하고 해당 텍스트가 긍정적인지 부정적인지 예측)
def infer(text): # 주어진 텍스트에 대해 예측을 수행하는 함수
    model.eval() # 모델을 평가모드로 설정
    tokenized_text = tokenizer(text) # 입력 텍스트를 토큰화
    numerical_text = torch.tensor(vocab(tokenized_text)).unsqueeze(0).to(device) # 토큰화된 텍스트를 인덱스 숫자로 변환한 후 텐서로 변환, unsqueeze로 배치 차원 추가
    with torch.no_grad(): # 그래디언트 계산 없이 예측 수행
        output = model(numerical_text).squeeze() # 모델을 통해 예측 수행 및 차원 축소
        predicted = torch.sigmoid(output).item() > 0.5 # 시그모이드 함수를 적용하고 0.5를 기준으로 이진분류
        return int(predicted) # 예측 결과를 정수로 변환

example_text = "아 이거 좋다" # 예시 텍스트 정의
prediction = infer(example_text) # 예시 텍스트에 대한 예측 수행
print(f'Predicted label for "{example_text}": {prediction}') # 예측 결과 출력


Predicted label for "아 이거 좋다": 0


In [ ]:
%cd /content/nsmc # 작업 디렉토리를 /content/nsmc로 변경

/content/nsmc


In [ ]:
import pandas as pd # 데이터처리 라이브러리
from sklearn.model_selection import train_test_split # 데이터분할 라이브러리
from sklearn.preprocessing import LabelEncoder # 레이블 인코딩을 위한 클래스
from torchtext.data.utils import get_tokenizer # 토크나이저 함수 임포트
from torchtext.vocab import build_vocab_from_iterator # 어휘구축을 위한 함수
import torch # 파이토치 라이브러리
import torch.nn as nn # 신경망 모듈
import torch.optim as optim # 최적화 알고리즘
from torch.utils.data import Dataset, DataLoader # 데이터셋 및 데이터로더 클래스
from sklearn.metrics import accuracy_score # 정확도 계산을 위한 함수
import warnings # 경고 메시지 제어를 위한 모듈
import torchtext # torchtext 라이브러리

# torchtext deprecation warning 경고메시지 무시
torchtext.disable_torchtext_deprecation_warning()

# 데이터 로딩
df = pd.read_csv('ratings_train.txt', delimiter='\t') # 탭으로 구분된 텍스트파일을 데이터프레임으로 읽기
df.columns = ['id', 'document', 'label'] # 데이터프레임 열 이름 지정

# NaN 값을 빈 문자열로 변환
df['document'] = df['document'].astype(str) # 'document'열을 문자열 타입으로 변환

# 레이블 인코딩
label_encoder = LabelEncoder() # LabelEncoder 객체 생성
df['label'] = label_encoder.fit_transform(df['label']) # 레이블을 숫자로 인코딩

# 데이터 분할
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42) # 훈련데이터와 검증데이터로 분할

# 토크나이저와 단어 집합 구축
tokenizer = get_tokenizer('basic_english') # 기본 영어 토크나이저 가져오기

def yield_tokens(data_iter): # 토큰 생성기 함수 정의
    for text in data_iter:
        yield tokenizer(text) # 각 텍스트에 대해 토큰화 수행

vocab = build_vocab_from_iterator(yield_tokens(df['document']), specials=['<unk>']) # 어휘구축
vocab.set_default_index(vocab['<unk>']) # 알수없는 토큰에 대한 기본 인덱스 설정

# 데이터셋 클래스
class ReviewDataset(Dataset):
    def __init__(self, dataframe, vocab, tokenizer): # 초기화 메서드
        self.data = dataframe # 데이터프레임 저장
        self.vocab = vocab # 어휘 객체 저장
        self.tokenizer = tokenizer # 토크나이저 저장

    def __len__(self): # 데이터셋의 길이 반환 메서드
        return len(self.data)

    def __getitem__(self, idx): # 인덱스로 항목 접근 메서드
        text = self.data.iloc[idx]['document'] # 텍스트 가져오기
        label = self.data.iloc[idx]['label'] # 레이블 가져오기
        tokenized_text = self.tokenizer(text) # 텍스트 토큰화
        numerical_text = torch.tensor(self.vocab(tokenized_text)) # 토큰을 숫자로 변환
        return numerical_text, torch.tensor(label) # 숫자화된 텍스트와 레이블 반환

train_dataset = ReviewDataset(train_df, vocab, tokenizer) # 훈련 데이터셋 생성
valid_dataset = ReviewDataset(valid_df, vocab, tokenizer) # 검증 데이터셋 생성

def collate_fn(batch): # 배치 처리 함수 정의
    texts, labels = zip(*batch) # 텍스트와 레이블 분리
    texts = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True) # 텍스트 패딩
    labels = torch.stack(labels) # 레이블을 텐서로 변환
    return texts, labels # 처리된 텍스트와 레이블 반환

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn) # 훈련 데이터 로더 생성
valid_loader = DataLoader(valid_dataset, batch_size=64, collate_fn=collate_fn) # 검증 데이터 로더 생성

In [ ]:
# LSTM 모델 정의
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, num_layers=1): # 초기화 메서드
        super(LSTMClassifier, self).__init__() # 상위 클래스 초기화
        self.embedding = nn.Embedding(vocab_size, embed_size) # 임베딩 레이어
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True) # LSTM fpdldj
        self.fc = nn.Linear(hidden_size, output_size) # 완전연결 레이어

    def forward(self, x): # 순전파 메서드
        x = self.embedding(x) # 임베딩 적용
        h, _ = self.lstm(x) # LSTM 통과
        out = self.fc(h[:, -1, :]) # 마지막 시퀀스 출력에 대해 완전연결 레이어 적용
        return out # 최종 출력 반환

# 하이퍼파라미터 설정
embed_size = 100 # 임베딩 차원
hidden_size = 128 # LSTM 은닉 상태 차원
output_size = len(label_encoder.classes_) # 출력 클래스 수
num_layers = 1 # LSTM 레이어 수
learning_rate = 0.001 # 학습률
num_epochs = 5 # 훈련 에포크

# 모델, 손실 함수, 옵티마이저 초기화
model = LSTMClassifier(len(vocab), embed_size, hidden_size, output_size, num_layers).to("cuda") # 모델 생성 및 GPU로 이동
criterion = nn.CrossEntropyLoss() # 손실함수 정의
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # 옵티마이저 정의

# 훈련 함수
def train():
  model.train() # 모델을 훈련모드로 설정
  for epoch in range(num_epochs): # 지정된 에포크수만큼 반복
    for inputs, labels in train_loader: # 배치 데이터 반복
      inputs, labels = inputs.to('cuda'), labels.to('cuda') # 데이터를 GPU로 이동
      optimizer.zero_grad() # 그래디언트 초기화
      outputs = model(inputs) # 순전파
      loss = criterion(outputs, labels) # 손실계산
      loss.backward() # 역전파
      optimizer.step() # 파라미터 업데이트
    print(f'Epoch[{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') # 에포크마다 손실 출력

# 검증 함수
def evaluate(loader):
  model.eval() # 모델을 평가모드로 설정
  all_labels = [] # 실제 레이블을 저장할 리스트
  all_preds = [] # 예측 레이블을 저장할 리스트
  with torch.no_grad(): # 그래디언트 계산 비활성화
    for inputs, labels in loader: # 배치 데이터 반복
      inputs, labels = inputs.to('cuda'), labels.to('cuda') # 데이터를 GPU로 이동
      outputs = model(inputs) # 순전파
      _, preds = torch.max(outputs, 1) # 예측 클래스 선택
      all_labels.extend(labels.cpu().numpy()) # 실제 레이블 저장
      all_preds.extend(preds.cpu().numpy()) 예측 레이블 저장
    return accuracy_score(all_labels, all_preds) # 정확도 계산 및 반환

# 모델 훈련 및 검증
train() # 모델훈련 실행
valid_accuracy = evaluate(valid_loader) # 검증 데이터로 정확도 평가
print(f'Validation Accuracy: {valid_accuracy:.4f}') # 검증 정확도 출력

# 인퍼런스 예제
def infer(text):
  model.eval() # 모델을 평가모드로 설정
  tokenized_text = tokenizer(text) # 입력 텍스트 토큰화
  numerical_text = torch.tensor(vocab(tokenized_text)).unsqueeze(0).to('cuda') # 토큰을 숫자로 변환하고 GPU로 이동
  with torch.no_grad(): # 그래디언트 계산 비활성화
    output = model(numerical_text) # 순전파
    _, predicted = torch.max(output, 1) # 예측 클래스 선택
    return label_encoder.inverse_transform(predicted.cpu().numpy())[0] # 예측 레이블을 원래 형태로 변환하여 반환

example_text = "아 이거 좋다" # 예시 텍스트
prediction = infer(example_text) # 예시 텍스트에 대한 예측 수행
print(f'Predicted label for "{example_text}": {prediction}') # 예측 결과 출력

Epoch[1/5], Loss: 0.4897
Epoch[2/5], Loss: 0.3563
Epoch[3/5], Loss: 0.4134
Epoch[4/5], Loss: 0.0787
Epoch[5/5], Loss: 0.1638
Validation Accuracy: 0.7906
Predicted label for "아 이거 좋다": 1
